In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.14.0 pyspark-shell'

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Streamming_assignment").getOrCreate()

:: loading settings :: url = jar:file:/usr/lib/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.databricks#spark-xml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4d6e5af1-fe3d-4e3f-a852-ad2ffd542f41;1.0
	confs: [default]
	found com.databricks#spark-xml_2.12;0.14.0 in central
	found commons-io#commons-io;2.8.0 in central
	found org.glassfish.jaxb#txw2;2.3.4 in central
	found org.apache.ws.xmlschema#xmlschema-core;2.2.5 in central
:: resolution report :: resolve 320ms :: artifacts dl 8ms
	:: modules in use:
	com.databricks#spark-xml_2.12;0.14.0 from central in [default]
	commons-io#commons-io;2.8.0 from central in [default]
	org.apache.ws.xmlschema#xmlschema-core;2.2.5 from central in [default]
	org.glassfish.jaxb#txw2;2.3.4 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|

In [3]:
!hdfs dfs -ls /user/root

Found 14 items
drwxr-xr-x   - root   hadoop          0 2022-05-01 16:04 /user/root/.sparkStaging
drwxr-xr-x   - root   hadoop          0 2022-05-01 15:47 /user/root/commits
drwxr-xr-x   - saisur hadoop          0 2022-05-01 15:50 /user/root/h2p3
drwxr-xr-x   - saisur hadoop          0 2022-05-01 15:59 /user/root/h2p3_m
-rw-r--r--   2 root   hadoop         45 2022-05-01 15:47 /user/root/metadata
drwxr-xr-x   - root   hadoop          0 2022-05-01 15:47 /user/root/offsets
drwxr-xr-x   - root   hadoop          0 2022-04-30 01:35 /user/root/p1t2_large
drwxr-xr-x   - root   hadoop          0 2022-04-30 17:21 /user/root/p1t2_small
drwxr-xr-x   - root   hadoop          0 2022-04-30 03:30 /user/root/p1t2_test_split
drwxr-xr-x   - root   hadoop          0 2022-04-30 18:53 /user/root/pagerank_large
drwxr-xr-x   - root   hadoop          0 2022-04-30 17:47 /user/root/pagerank_small
drwxr-xr-x   - root   hadoop          0 2022-04-30 00:10 /user/root/part2_testq2
drwxr-xr-x   - root   hadoop         

In [4]:
from pyspark.sql.types import StructType

userSchema = StructType().add("article", "string").add("rank", "double")
csvDF = spark.readStream \
    .option("sep", "\t") \
    .schema(userSchema) \
    .csv("hdfs:/user/root/pagerank_large")

In [5]:
df = csvDF.select('article', 'rank').where("rank > 0.5")

In [6]:
# hadoop fs -mkdir /user/root/h2p3

In [7]:
df.writeStream \
    .format("csv") \
    .option("delimiter", "\t") \
    .option("checkpointLocation", "hdfs:/user/root/h2p3_m/") \
    .option("path", "/user/root/h2p3_m/") \
    .start()

22/05/01 16:04:53 WARN org.apache.spark.sql.streaming.StreamingQueryManager: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [11]:
df.createOrReplaceTempView("updates")
counts = spark.sql("select count(*) from updates") 

In [ ]:
query = counts \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()

query.awaitTermination()

22/05/01 16:08:28 WARN org.apache.spark.sql.streaming.StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-4f8a7c4f-c6b2-4a46-ad93-3095f007f89c. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/05/01 16:08:28 WARN org.apache.spark.sql.streaming.StreamingQueryManager: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+--------+
|count(1)|
+--------+
| 1634800|
+--------+

